In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#Score: 0.91186
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy
/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv


In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [9]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/audio-binary-classification/train_data.npy")
X_test = np.load("/kaggle/input/audio-binary-classification/test_data.npy")
y_train = pd.read_csv('/kaggle/input/audio-binary-classification/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = np.reshape(X_train, (-1, 210,210))
X_test = np.reshape(X_test, (X_test.shape[0], 210,210))


In [10]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.20, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=True))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

Build LSTM RNN model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 210, 128)          173568    
_________________________________________________________________
lstm_4 (LSTM)                (None, 210, 64)           49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 235,425
Trainable params: 235,425
Non-trainable params: 0
_________________________________________________________________


In [11]:

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 100, epochs = 50)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_LSTM_128_64_32.csv", index=False)


Epoch 1/50
3999/3999 [==============================] - 49s 12ms/step - loss: 0.3834 - acc: 0.8617
Epoch 2/50
3999/3999 [==============================] - 47s 12ms/step - loss: 0.2641 - acc: 0.8730
Epoch 3/50
3999/3999 [==============================] - 46s 11ms/step - loss: 0.2521 - acc: 0.8772
Epoch 4/50
3999/3999 [==============================] - 46s 11ms/step - loss: 0.2454 - acc: 0.8775
Epoch 5/50
3999/3999 [==============================] - 46s 11ms/step - loss: 0.2389 - acc: 0.8785
Epoch 6/50
3999/3999 [==============================] - 46s 11ms/step - loss: 0.2334 - acc: 0.8797
Epoch 7/50
3999/3999 [==============================] - 45s 11ms/step - loss: 0.2348 - acc: 0.8780
Epoch 8/50
3999/3999 [==============================] - 45s 11ms/step - loss: 0.2296 - acc: 0.8787
Epoch 9/50
3999/3999 [==============================] - 47s 12ms/step - loss: 0.2226 - acc: 0.8840
Epoch 10/50
3999/3999 [==============================] - 46s 11ms/step - loss: 0.2189 - acc: 0.8817
Epoch 11/